In [9]:
%%time
from pulp import *

n = 4
c = [[0, 13, 154, 220],
     [13, 0, 335, 25],
     [154, 335, 0, 320],
     [220, 25, 320, 0]]


prob = LpProblem("MTZ", LpMinimize)

x = [[LpVariable(f"x_{i}_{j}", cat=LpBinary) for j in range(n)] for i in range(n)]
u = [LpVariable(f"u_{i}", lowBound=2, upBound=n, cat=LpInteger) for i in range(n)]

prob += lpSum(c[i][j] * x[i][j] for i in range(n) for j in range(n) if i != j)

for i in range(n):
    prob += lpSum(x[i][j] for j in range(n) if j != i) == 1  # One edge from each node in the path
    prob += lpSum(x[j][i] for j in range(n) if j != i) == 1  # One edge to each node in the path

for i in range(1, n):
    for j in range(1, n):
        if i != j:
            prob += u[i] - u[j] + 1 <= (n - 1) * (1 - x[i][j])

for i in range(2, n):
    prob += 2 <= u[i] <= n  # Bounds on u_i
    for j in range(2, n):
        if i != j:
            prob += u[i] - u[j] + 1 <= (n - 1) * (1 - x[i][j])

prob.solve()

print("Status:", LpStatus[prob.status])
print("Objective function value:", value(prob.objective))

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --cpxlp /tmp/e9b59a869534465b8d73485c11f08289-pulp.lp -o /tmp/e9b59a869534465b8d73485c11f08289-pulp.sol
Reading problem data from '/tmp/e9b59a869534465b8d73485c11f08289-pulp.lp'...
18 rows, 15 columns, 50 non-zeros
15 integer variables, 12 of which are binary
45 lines were read
GLPK Integer Optimizer 5.0
18 rows, 15 columns, 50 non-zeros
15 integer variables, 12 of which are binary
Preprocessing...
16 rows, 15 columns, 48 non-zeros
15 integer variables, 12 of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  3.000e+00  ratio =  3.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 15
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
16 rows, 15 columns, 48 non-zeros
      0: obj =   1.110000000e+03 inf =   1.000e+01 (7)
      4: obj =   7.853333333e+02 inf =   0.000e+00 (0)
*     8: obj =   4.606666667e+02 inf =   1.110e-16 (0)
OPTIMAL LP SOLUT

In [10]:
%%time
from pulp import *

n = 4
c = [[0, 13, 154, 220],
     [13, 0, 335, 25],
     [154, 335, 0, 320],
     [220, 25, 320, 0]]


prob = LpProblem("DFJ", LpMinimize)

x = [[LpVariable(f"x_{i}_{j}", cat=LpBinary) for j in range(n)] for i in range(n)]

prob += lpSum(c[i][j] * x[i][j] for i in range(n) for j in range(n) if i != j)

for i in range(n):
    prob += lpSum(x[i][j] for j in range(n) if j != i) == 1  # One edge from each node in the path
    prob += lpSum(x[j][i] for j in range(n) if j != i) == 1  # One edge to each node in the path

#subtour elimination constraint
for subset_size in range(2, n):
    subsets = itertools.combinations(range(n), subset_size)
    for subset in subsets:
        prob += lpSum(x[i][j] for i in subset for j in subset if i != j) <= len(subset) - 1

prob.solve()

print("Status:", LpStatus[prob.status])
print("Objective function value:", value(prob.objective))

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --cpxlp /tmp/663f4cb2bee94f24816fc83ed94c75e6-pulp.lp -o /tmp/663f4cb2bee94f24816fc83ed94c75e6-pulp.sol
Reading problem data from '/tmp/663f4cb2bee94f24816fc83ed94c75e6-pulp.lp'...
18 rows, 12 columns, 60 non-zeros
12 integer variables, all of which are binary
37 lines were read
GLPK Integer Optimizer 5.0
18 rows, 12 columns, 60 non-zeros
12 integer variables, all of which are binary
Preprocessing...
18 rows, 12 columns, 60 non-zeros
12 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 17
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
18 rows, 12 columns, 60 non-zeros
      0: obj =   1.110000000e+03 inf =   5.000e+00 (5)
      1: obj =   7.340000000e+02 inf =   0.000e+00 (0)
*     5: obj =   5.120000000e+02 inf =   0.000e+00 (0)
OPTIMAL LP SO